# Notebook 7: Creating digital signature with public key

### 1. Signing transactions to create a digital signature appended with public key

In [1]:
from ecdsa import SigningKey, SECP256k1

sk = SigningKey.generate(curve=SECP256k1)
vk = sk.get_verifying_key()

print("sk: ", sk.to_string())
print("vk: ", vk.to_string())

sk:  b')\xea2Q\xd4\xb4su\xf7pGS"h\x18\x83\xe7}7\xcfrp\xf2&\x86\x1e\xb9\xc94\xcd!\x9f'
vk:  b'\x02(\x14O\x88\xddxU\xe6\xcd\xf1\x9c\xc3F\xb9\xd2\x90\xdb\xf3\xcd5\x0b\\\x06T\n\xd9h\xa9\tp*\xd0\x0b\x8bD)?\xc7\xcd\x07\x89\xcd6}!!\x96\xa8$xL}\x1c\xef\xf5\xd0\xcdS\xd54*\x18\xa9'


In [2]:
import json 

txns = {"Jimmy": -50, "Alice": 50}
sig = sk.sign(str.encode(json.dumps(txns)))

print("Sig: ", sig)

# Combine signature with 
sig_vk = (sig + vk.to_string()).hex()
print("\nSig_vk:\n", sig_vk)

Sig:  b'\x92~GS\xedlZk3\xb6\xc1I\xaf\xae\xcbv#z\xce1\x03\xa8x\xb04O\xd6\xb5`\xa6Z\xe7G[\xae\xa3w\xd8uo\xf6\r\x94\x84+\xf2 \xba\xf7\xdc\xe0s^\xe6\x04\x18PO3DHb\xc8|'

Sig_vk:
 927e4753ed6c5a6b33b6c149afaecb76237ace3103a878b0344fd6b560a65ae7475baea377d8756ff60d94842bf220baf7dce0735ee60418504f33444862c87c0228144f88dd7855e6cdf19cc346b9d290dbf3cd350b5c06540ad968a909702ad00b8b44293fc7cd0789cd367d212196a824784c7d1ceff5d0cd53d5342a18a9


In [3]:
# Receiver: 
# - Receives signature in hexidecimal format 
# - Converts to bytes, seperates 

def parse_signature(hex_str): 
    sig_plus_vk = bytes.fromhex(hex_str)
    return sig_plus_vk[:64], sig_plus_vk[64:]

actual_sig, actual_vk = parse_signature(sig_vk)

print("Sig: ", actual_sig)
print("vk: ", actual_vk)

Sig:  b'\x92~GS\xedlZk3\xb6\xc1I\xaf\xae\xcbv#z\xce1\x03\xa8x\xb04O\xd6\xb5`\xa6Z\xe7G[\xae\xa3w\xd8uo\xf6\r\x94\x84+\xf2 \xba\xf7\xdc\xe0s^\xe6\x04\x18PO3DHb\xc8|'
vk:  b'\x02(\x14O\x88\xddxU\xe6\xcd\xf1\x9c\xc3F\xb9\xd2\x90\xdb\xf3\xcd5\x0b\\\x06T\n\xd9h\xa9\tp*\xd0\x0b\x8bD)?\xc7\xcd\x07\x89\xcd6}!!\x96\xa8$xL}\x1c\xef\xf5\xd0\xcdS\xd54*\x18\xa9'


### 2. Function to generate bitcoin address from public key

In [6]:
from Crypto.Hash import SHA256
from Crypto.Hash.RIPEMD import RIPEMD160Hash
import base58

def generate_bitcoin_address(pubkey):
    
    if len(pubkey) != 64: 
        raise Exception("Please enter a")

    hash_1 = SHA256.new(pubkey).hexdigest() # Stage 1 
    r160_hash_obj = RIPEMD160Hash() # Stage 2
    r160_hash_obj.update(hash_1)    
    hash_2 = r160_hash_obj.hexdigest()
    hash_3 = "00" + hash_2 # Stage 3 

    hash_4 = SHA256.new(str.encode(hash_3)).digest().hex() # Stage 4
    hash_5 = SHA256.new(str.encode(hash_4)).digest() # Stage 5
    checksum = hash_5[:4].hex() # Stage 6
    
    bit_address = hash_3 + checksum # Stage 7
    return base58.b58encode(bytes.fromhex(bit_address)) # Stage 8

public_addr = generate_bitcoin_address(vk.to_string())

print("Public address: ", public_addr)

Public address:  1HBgFTJZ9VAnb4Cr4DYqH99cZngffzQiMd


In [11]:
vk.to_string()

b'\x02(\x14O\x88\xddxU\xe6\xcd\xf1\x9c\xc3F\xb9\xd2\x90\xdb\xf3\xcd5\x0b\\\x06T\n\xd9h\xa9\tp*\xd0\x0b\x8bD)?\xc7\xcd\x07\x89\xcd6}!!\x96\xa8$xL}\x1c\xef\xf5\xd0\xcdS\xd54*\x18\xa9'

### 3. Check the hash to confirm verification keys

In [7]:
if public_addr == generate_bitcoin_address(actual_vk): 
    print("The public key is valid!")
else: 
    print("This key is invalid")


The public key is valid!


### 4. Verify the contents of the document

In [8]:
from ecdsa import SigningKey, VerifyingKey, NIST384p

txns = {"Jimmy": -50, "Alice": 50}
unicode_json_txns = str.encode(json.dumps(txns))

vk_obj = VerifyingKey.from_string(actual_vk, curve=SECP256k1)

vk_obj.verify(actual_sig, unicode_json_txns)

True

This is how you make do it!!